In [1]:
import json
import sqlite3
from sqlite3 import Error

In [2]:
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print(f'Успешное подключение к базе по пути {path}')
    except Error as e:
        print(f'Ошибка {e}')
    return connection

In [3]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print('Запрос успешно выполнен')
    except Error as e:
        print(f'Ошибка {e}')

In [4]:
connection = create_connection('DB/DB_V1.sqlite')

Успешное подключение к базе по пути DB/DB_V1.sqlite


In [5]:
# Общая таблица рег номеров, от неё пляшем на 4 таблицы с базами и на поисковую таблицу
# Ключи вяжем как родительскую к 4 базам, тогда каждая сможет обратиться к ней
create_main_table = '''
CREATE TABLE IF NOT EXISTS main (
    reg_num TEXT PRIMARY KEY,
    _index TEXT NOT NULL,
    name TEXT NOT NULL,
    annotation TEXT NOT NULL,
    full_text TEXT NOT NULL,
    region TEXT,
    reg_date TEXT NOT NULL,
    search_date TEXT NOT NULL,

    author_name TEXT,
    author_surname TEXT NOT NULL,
    author_patronymic TEXT,
    position TEXT,

    org_name TEXT,
    org_short_name TEXT,

    org_supervisor_name TEXT,
    org_supervisor_surname TEXT,
    org_supervisor_patronymic TEXT,
    org_supervisor_position TEXT,

    other_org_name TEXT,
    other_org_short_name TEXT,

    priority TEXT,
    priority_sub TEXT,

    FOREIGN KEY (region) REFERENCES region (region) ON DELETE CASCADE,    
    FOREIGN KEY (_index) REFERENCES _index (_index) ON DELETE CASCADE,        
    FOREIGN KEY (priority) REFERENCES priority (letter) ON DELETE CASCADE,
    FOREIGN KEY (priority_sub) REFERENCES priority_sub (word) ON DELETE CASCADE,
    FOREIGN KEY (search_date) REFERENCES day (date)
)
'''
execute_query(connection, create_main_table)

create_priority_table = '''
CREATE TABLE IF NOT EXISTS priority (
    letter TEXT PRIMARY KEY,
    word TEXT
)
'''
execute_query(connection, create_priority_table)

create_priority_sub_table = '''
CREATE TABLE IF NOT EXISTS priority_sub (
    id INTEGER PRIMARY KEY, 
    word TEXT,
    sub_word TEXT,
    FOREIGN KEY (word) REFERENCES priority (word)
)
'''
execute_query(connection, create_priority_sub_table)


create_keysearch_table = '''
CREATE TABLE IF NOT EXISTS keysearch (
    word TEXT PRIMARY KEY
)
'''
execute_query(connection, create_keysearch_table)

create_all_words_table = '''
CREATE TABLE IF NOT EXISTS keysearch_words (
    id INTEGER PRIMARY KEY,
    word TEXT NOT NULL,
    reg_num TEXT NOT NULL,
    FOREIGN KEY (reg_num) REFERENCES main (reg_num)
    FOREIGN KEY (word) REFERENCES keysearch (word)

)
'''
execute_query(connection, create_all_words_table)

create_index_table = '''
CREATE TABLE IF NOT EXISTS _index (
    _index TEXT PRIMARY KEY
)
'''
fill_index_table = '''
INSERT INTO _index (_index) VALUES
("rids"), ("dissertations"), ("ikrbses"), ("nioktrs"), ("rnfs")
'''
execute_query(connection, create_index_table)
execute_query(connection, fill_index_table)

create_region_table = '''
CREATE TABLE IF NOT EXISTS region (
    region TEXT PRIMARY KEY    
)
'''
execute_query(connection, create_region_table)

create_nioktr_budget_types_table = '''
CREATE TABLE IF NOT EXISTS nioktr_budget_types (
    name TEXT PRIMARY KEY
)
'''
execute_query(connection, create_nioktr_budget_types_table)


create_year_table = '''
CREATE TABLE IF NOT EXISTS year (
    year TEXT PRIMARY KEY    
)
'''
execute_query(connection, create_year_table)

create_mounth_table = '''
CREATE TABLE IF NOT EXISTS mounth (
    id INTEGER PRIMARY KEY,
    year TEXT NOT NULL,
    mounth TEXT NOT NULL,

    FOREIGN KEY (year) REFERENCES year (year)
)
'''
execute_query(connection, create_mounth_table)

create_day_table = '''
CREATE TABLE IF NOT EXISTS day (
    id INTEGER PRIMARY KEY,
    mounth TEXT NOT NULL,
    date TEXT NOT NULL,

    FOREIGN KEY (mounth) REFERENCES mounth (mounth)   
)
'''
execute_query(connection, create_day_table)


create_rubrics_lvl1_table = '''
CREATE TABLE IF NOT EXISTS rubrics_lvl1 (
    rubrics_lvl1 TEXT PRIMARY KEY    
)
'''
execute_query(connection, create_rubrics_lvl1_table)

create_rubrics_lvl2_table = '''
CREATE TABLE IF NOT EXISTS rubrics_lvl2 (
    id INTEGER PRIMARY KEY,
    rubrics_lvl1 TEXT NOT NULL,
    rubrics_lvl2 TEXT NOT NULL,

    FOREIGN KEY (rubrics_lvl1) REFERENCES rubrics_lvl1 (rubrics_lvl1)
)
'''
execute_query(connection, create_rubrics_lvl2_table)

create_rubrics_lvl3_table = '''
CREATE TABLE IF NOT EXISTS rubrics_lvl3 (
    id INTEGER PRIMARY KEY,
    rubrics_lvl2 TEXT NOT NULL,
    rubrics_lvl3 TEXT NOT NULL,
    name TEXT,

    FOREIGN KEY (rubrics_lvl2) REFERENCES rubrics_lvl2 (rubrics_lvl2)   
)
'''
execute_query(connection, create_rubrics_lvl3_table)



# Ниже дочерние таблицы
create_keywords_table = '''
CREATE TABLE IF NOT EXISTS keywords (
    id INTEGER PRIMARY KEY,
    reg_num TEXT NOT NULL,
    keyword TEXT NOT NULL,
    FOREIGN KEY (reg_num) REFERENCES main (reg_num) 
)
'''
execute_query(connection, create_keywords_table)

create_oecds_table = '''
CREATE TABLE IF NOT EXISTS oecds (
    id INTEGER PRIMARY KEY,
    reg_num TEXT NOT NULL,
    name TEXT NOT NULL,
    code TEXT NOT NULL,
    FOREIGN KEY (reg_num) REFERENCES main (reg_num) 
)
'''
execute_query(connection, create_oecds_table)

create_rubrics_table = '''
CREATE TABLE IF NOT EXISTS rubrics (
    id INTEGER PRIMARY KEY,
    reg_num TEXT NOT NULL,
    code TEXT NOT NULL,
    udk TEXT NOT NULL,
    FOREIGN KEY (reg_num) REFERENCES main (reg_num),
    FOREIGN KEY (code) REFERENCES rubrics_lvl3 (rubrics_lvl3)
)
'''
execute_query(connection, create_rubrics_table)

# Отсюда таблицы по индексам
#   Ниже РНФ
create_rnf_table = '''
CREATE TABLE IF NOT EXISTS rnfs (
    reg_num TEXT PRIMARY KEY,
    period_of_execution TEXT,
    contest TEXT,
    area_of_knowledge TEXT,
    expected_results TEXT,
    reporting_materials TEXT,    
    FOREIGN KEY (reg_num) REFERENCES main (reg_num)
)
'''
execute_query(connection, create_rnf_table)

#   Ниже Дисертация
create_dissertations_table = '''
CREATE TABLE IF NOT EXISTS dissertations (
    reg_num TEXT PRIMARY KEY,
    type TEXT,
    report_type TEXT,
    degree_pursued TEXT,
    chairman_name TEXT,
    chairman_surname TEXT,
    chairman_patronymic TEXT,
    chairman_position TEXT,
    chairman_degree TEXT,
    speciality_codes TEXT,

    protection_date TEXT,
    tables_count INTEGER,
    pictures_count INTEGER,
    bibliography INTEGER,
    applications_count INTEGER,
    pages_count INTEGER,
    sources_count INTEGER,
    books_count INTEGER,    
    FOREIGN KEY (reg_num) REFERENCES main (reg_num)
)
'''
execute_query(connection, create_dissertations_table)

create_supervisors_table = '''
CREATE TABLE IF NOT EXISTS supervisors (
    id INTEGER PRIMARY KEY,
    reg_num TEXT,
    fio TEXT,
    degree TEXT,
    speciality_name TEXT,
    speciality_code TEXT,        
    FOREIGN KEY (reg_num) REFERENCES dissertations (reg_num)
)
'''
execute_query(connection, create_supervisors_table)

create_opponents_table = '''
CREATE TABLE IF NOT EXISTS opponents (
    id INTEGER PRIMARY KEY,
    reg_num TEXT,
    fio TEXT,
    degree TEXT,
    speciality_name TEXT,
    speciality_code TEXT,        
    FOREIGN KEY (reg_num) REFERENCES dissertations (reg_num)
)
'''
execute_query(connection, create_opponents_table)


#   Ниже РИД
create_rids_table = '''
CREATE TABLE IF NOT EXISTS rids (
    reg_num TEXT PRIMARY KEY,
    rid_type TEXT,
    expected TEXT,
    nioktr TEXT,
    using_ways TEXT,
    FOREIGN KEY (reg_num) REFERENCES main (reg_num)
)
'''
execute_query(connection, create_rids_table)


#   НИЖЕ ИКРБСЕС
create_ikrbses_table = '''
CREATE TABLE IF NOT EXISTS ikrbses (
    reg_num TEXT PRIMARY KEY,
    nioktr TEXT,
    approve_date TEXT,
    applications_count INTEGER,
    books_count INTEGER,
    pages_count INTEGER,
    tables_count INTEGER,
    pictures_count INTEGER,
    publication_count INTEGER,
    bibliography INTEGER,   
    FOREIGN KEY (reg_num) REFERENCES main (reg_num)
)
'''
execute_query(connection, create_ikrbses_table)

create_rids_or_ikrnses_authors_table = '''
CREATE TABLE IF NOT EXISTS authors (
    id INTEGER PRIMARY KEY,
    reg_num TEXT NOT NULL,
    name TEXT NOT NULL,
    surname TEXT NOT NULL,
    patronymic TEXT NOT NULL,
    description TEXT NOT NULL,
    FOREIGN KEY (reg_num) REFERENCES main (reg_num)
)
'''
execute_query(connection, create_rids_or_ikrnses_authors_table)

#   НИЖЕ НИОКТР
create_nioktr_table = '''
CREATE TABLE IF NOT EXISTS nioktrs (
    reg_num TEXT PRIMARY KEY,
    bases TEXT,
    types TEXT,
    start_date TEXT,
    end_date TEXT,
    contract_number TEXT,
    contract_date TEXT,    
    FOREIGN KEY (reg_num) REFERENCES main (reg_num)
)
'''
execute_query(connection, create_nioktr_table)

create_nioktr_budgets_table = '''
CREATE TABLE IF NOT EXISTS nioktr_budgets (
    id INTEGER PRIMARY KEY,
    reg_num TEXT,
    name TEXT,
    funds REAL,
    FOREIGN KEY (reg_num) REFERENCES nioktrs (reg_num),
    FOREIGN KEY (name) REFERENCES nioktr_budget_types (name)
)
'''
execute_query(connection, create_nioktr_budgets_table)

create_nioktr_critical_technologies_table = '''
CREATE TABLE IF NOT EXISTS nioktr_technology (
    id INTEGER PRIMARY KEY,
    reg_num TEXT,
    name TEXT,
    FOREIGN KEY (reg_num) REFERENCES nioktrs (reg_num)
)
'''
execute_query(connection, create_nioktr_critical_technologies_table)

create_nioktr_priority_directions_table = '''
CREATE TABLE IF NOT EXISTS nioktr_priority (
    id INTEGER PRIMARY KEY,
    reg_num TEXT,
    name TEXT,
    FOREIGN KEY (reg_num) REFERENCES nioktrs (reg_num)
)
'''
execute_query(connection, create_nioktr_priority_directions_table)

create_nioktr_coexecutors_table = '''
CREATE TABLE IF NOT EXISTS nioktr_coexecutors(
    id INTEGER PRIMARY KEY,
    reg_num TEXT,
    name TEXT,
    short_name TEXT,
    FOREIGN KEY (reg_num) REFERENCES nioktrs (reg_num)
) 
'''
execute_query(connection, create_nioktr_coexecutors_table)
connection.close()

Запрос успешно выполнен
Запрос успешно выполнен
Запрос успешно выполнен
Запрос успешно выполнен
Запрос успешно выполнен
Запрос успешно выполнен
Запрос успешно выполнен
Запрос успешно выполнен
Запрос успешно выполнен
Запрос успешно выполнен
Запрос успешно выполнен
Запрос успешно выполнен
Запрос успешно выполнен
Запрос успешно выполнен
Запрос успешно выполнен
Запрос успешно выполнен
Запрос успешно выполнен
Запрос успешно выполнен
Запрос успешно выполнен
Запрос успешно выполнен
Запрос успешно выполнен
Запрос успешно выполнен
Запрос успешно выполнен
Запрос успешно выполнен
Запрос успешно выполнен
Запрос успешно выполнен
Запрос успешно выполнен
Запрос успешно выполнен
Запрос успешно выполнен
Запрос успешно выполнен


In [6]:
connection.close()